<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/01-Pandas/A3-Advanced_Data_Manipulation_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pandas beyond SQL: Advanced Data Manipulation Techniques

Install the necessary libraries to connect to MySQL and to read Excel files

In [ ]:
# @title Setup and preliminaries

!pip3 install -U -q PyMySQL sqlalchemy

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Render our plots with high resolution
%config InlineBackend.figure_format = 'retina'

# Make the graphs a bit bigger
matplotlib.style.use(["seaborn-v0_8-talk", "seaborn-v0_8-ticks", "seaborn-v0_8-whitegrid"])

# Setting the default figure size for Pandas plots
pd.options.plotting.backend = 'matplotlib'
plt.rcParams['figure.figsize'] = [10, 3]

In [ ]:
# @title MySQL Setup

import os
from sqlalchemy import create_engine
from sqlalchemy import text

conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org',
    user = 'student',
    password = 'dwdstudent2025',
    db = 'doh_restaurants',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)

# This query returns back the restaurants in the DOH database
sql = '''
	SELECT R.CAMIS, R.DBA, R.BUILDING, R.STREET, R.ZIPCODE, R.BORO,
          R.CUISINE_DESCRIPTION, R.LATITUDE, R.LONGITUDE, R.NTA
		FROM doh_restaurants.restaurants R
'''

with engine.connect() as connection:
	restaurants = pd.read_sql(text(sql), con=connection)

# This query returns back the results of the inspections of each restaurant
sql = '''
	SELECT R.CAMIS, R.DBA, R.ZIPCODE, R.BORO, R.CUISINE_DESCRIPTION, R.NTA,
				 I.INSPECTION_DATE, I.INSPECTION_ID,
				 I.INSPECTION_TYPE, I.SCORE, I.GRADE
	FROM restaurants R
		JOIN inspections I ON I.CAMIS = R.CAMIS
'''

with engine.connect() as connection:
	inspections = pd.read_sql(text(sql), con=connection)

# Pivot Tables



## Simple pivot tables

[Pivot tables](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) is one of the most commonly used exploratory tools, and in Pandas they are extremely flexible.

In their simplest form, they can generate the same results as aggregate queries.

In [ ]:
rest_per_boro = pd.pivot_table(
    data=restaurants,
    aggfunc="count",  # we count... (the aggregation function)
    values="CAMIS",  # ...the different CAMIS numbers (what we aggregate)
    index="BORO",  # ..per BORO (and how we specify the groups, ie the variable that we use for dividing the data)
)


In [ ]:
# The full pivot table
rest_per_boro

Or, let's try to examine the average inspection score, broken down by cuisine.

In [ ]:
score_per_cuisine = pd.pivot_table(
    data=inspections,
    index="CUISINE_DESCRIPTION",  # specifies the rows
    values="SCORE",  # specifies the content of the cells
    aggfunc="mean",  # we calculate the average/mean score
)

In [ ]:
# The full pivot table
score_per_cuisine

In [ ]:
# Plotting the scores of the 10 highest scoring cuisines
(
    score_per_cuisine
    .sort_values('SCORE') # sort in asceding order of score
    .tail(10) # take the 10 entries in the largest values (at the botoom)
    .plot(kind='barh') # create a horizontal bar plot / we use horizontal bar for better label readability
)

### Exercise

Show the average inspection score broken down by neighborhood code (the `NTA` variable).

### Pivot Tables and Dates

In [ ]:
# Count the number of CAMIS values that appear on each date
pivot = pd.pivot_table(
    data=inspections,
    index="INSPECTION_DATE",  # specifies the rows
    values="CAMIS",  # specifies the content of the cells
    aggfunc="count",  # we ask to count how many different CAMIS values we see
)

In [ ]:
pivot

Now, let's plot this. By default, Pandas considers the "index" column to be the x-axis, and plots the rest.

In [ ]:
pivot.plot(kind = 'line', y = 'CAMIS', figsize=(10, 4))

In [ ]:
# We keep the last 100 entries and plot
pivot.tail(100).plot()

### Exercise

Now let's do an exercise,  instead of counting the number of inspections, we want to compute the average score assigned by the inspectors. Hint: We will need to change the `values` and the `aggfunc` parameters in the `pivot_table` function above.

In [ ]:
# your code here

#### Solution

In [ ]:
pivot = pd.pivot_table(
    data=inspections,
    index="INSPECTION_DATE",  # specifies the rows
    values="SCORE",  # specifies the content of the cells
    aggfunc="mean",  # compute the average SCORE
)

In [ ]:
pivot.plot(figsize=(10, 4))

### Changing date granularity

We can also use the [resample](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) command to change the frequency from one day, to, say, 7 days. Then we can compute, say, the average (`mean()`) for these days, or the total number (`sum()`) of inspections.

In [ ]:
pivot = pd.pivot_table(
    data=inspections,
    index="INSPECTION_DATE",  # specifies the rows
    values="CAMIS",  # specifies the content of the cells
    aggfunc="count",  # we ask to count how many different CAMIS values we see
)

pivot.resample("1W").sum()

Now, let's plot this. By default, Pandas considers the "index" column to be the x-axis, and plots the rest.

In [ ]:
# Plot the average number of inspections, over 7-day periods
pivot.resample("1W").sum().plot()

In [ ]:
# Plot the total number of inspections, over 1-month periods
pivot.resample("1M").sum().plot()

In [ ]:
plot = pivot.resample("7D").mean().plot()
plot.set_xlabel("Date of Inspection")
plot.set_ylabel("Average Number of Inspections (over a 7-day period)")
plot.set_title("Analysis of Number of Inspections over Time")

## Pivot Table with two (or more) variables

We would like to break down the results by borough, so we add the `column` parameter.

In [ ]:
# Show grades broken down by borough
pivot2 = pd.pivot_table(
    data=inspections,  #
    index="GRADE",
    columns="BORO",
    values="CAMIS",
    aggfunc="count"
)
pivot2

In [ ]:
# By default, the "index" becomes the x-axis and we plot all numeric columns
pivot2.plot(kind='bar')

## Advanced Pivot Tables

We can also add multiple attributes in the index and columns. It is also possible to have multiple aggregation functions, and we can even define our own aggregation functions.

In [ ]:
# We break down by BORO and GRADE, and also calculate
# inspections scores
pivot_advanced = pd.pivot_table(
    data=inspections,  #
    index="INSPECTION_DATE",
    columns=["BORO", "GRADE"],
    values="SCORE",
    aggfunc=["mean", "std"],
)

# Take the total number of inspections (unique and non-unique)
agg = pivot_advanced.resample("1M").mean()

# Show the last 5 entries and show the transpose (.T)
agg.tail().T

# Normalization, with application to Pivot Tables (and beyond)

In [ ]:
# Let's say that we want to normalize the values
# to account for the different number of inspections
# in each borough
pivot2.sum()

In [ ]:
# This command normalizes each column, by dividing with its sum:
pivot2 / pivot2.sum()

In [ ]:
normalized_by_borough = pivot2 / pivot2.sum()

# Percent of inspections per borough with a given grade
normalized_by_borough.plot(kind='bar')

#### Normalization and Transpose

In [ ]:
# This takes the transpose of the dataframe
pivot2.T

In [ ]:
# Plotting the transpose now uses the boroughs as the "x" axis
pivot2.T.plot(kind='barh')

In [ ]:
# Variation,
pivot2.T.plot(kind='barh', stacked=True)

In [ ]:
normalized_by_borough.T.plot(kind='barh', stacked=True).legend(frameon=True)



### Exercise

We now want to examine if different cuisines have different inspection scores. Compute the average inspection score by cuisine. Use the `sort_values()` command ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html)) to order cuisines by their inspection scores. Analyze further by breaking down the score over inspection dates, and plot.

#### Solution

In [ ]:
popular = (
    pd.pivot_table(
    data=inspections,
    index="CUISINE_DESCRIPTION",  # cuisines are the rows
    values="SCORE",  # we analyze the SCORE
    aggfunc=["mean","count"]  # compute the average SCORE
    )
    .sort_values(by=('count','SCORE'), ascending=False) # sort according to the popularity (count) of each cuisine
    .head(4) # keep the top-4 most popular cuisined
    .index # keep the names of the cuisines, which appear in the index
    .values # and retrieve their values
)

In [ ]:
# Select 3 columns (american, frenchn, chinese), compute the average score for a month
# and plot the results
(
    pd.pivot_table(
        data=inspections,
        index="INSPECTION_DATE",  # dates are the rows
        columns="CUISINE_DESCRIPTION",  # cuisines are the columns
        values="SCORE",  # we analyze the SCORE
        aggfunc="mean"  # compute the average SCORE
    )
    .filter(popular) # keep only the popular cuisines
    .resample("3M").mean() # convert granularity to monthly and taking average score
    .reset_index() # we take the INSPECTION_DATE out of the index to query and limit the x-axis
    .query("INSPECTION_DATE > '2021-01-01' ") # keep only dates after 2001
    .plot(
        kind = 'line',
        marker = 'o', markersize = 5, linewidth = 1,
        x = "INSPECTION_DATE",
        y = popular,
        figsize=(15, 4),
        title = "Average Score by Cuisine over Time"
      ).legend(
          loc='upper left',
      )
)

In [ ]:
# your code here

# Defining new columns -- `assign()` and `apply()`



### Using the `assign()` approach

The `assign` command applies a function to a dataframe and returns back a new dataframe with the new column(s).

In [ ]:
# We define functions that take as input a dataframe df
# and return back a new column.

# This function combines STREET/BUILDING/BORO/ZIPCODE columns into one address
#
def combine_address(df):
  return (df.BUILDING + ' ' + df.STREET + ', ' + df.BORO + ', NY ' + df.ZIPCODE).str.upper()


# This function computes the distance (in miles) from NYU,
# given the lat/lon of the other location
#
def distance(df):
  NYU_lon = -73.9962293
  NYU_lat = 40.7291527
  # The calculation below is simply the Pythagorean theorem.
  # The normalizing values "0.0146" and "0.0196"
  # are just for converting lat/lon differences to miles
  distance = ((df['LATITUDE']-NYU_lat)/0.0146)**2 + ((df['LONGITUDE']-NYU_lon)/0.0196)**2
  return np.sqrt(distance)



In [ ]:
# We now use the `assign` function to create two new columns
# using the logic in the functions above,
(
  restaurants
  .assign(
      distance_from_NYU = distance,
      address = combine_address
  )
  .filter(items = ['DBA','address','distance_from_NYU'])
)

In [ ]:
# And let's eliminate now duplicates and sort by distance
(
  restaurants
  .assign(
      distance_from_NYU = distance,
      address = combine_address
  )
  .filter(items = ['DBA','address','distance_from_NYU'])
  .query('distance_from_NYU > 0') # eliminates NaN values from distance_from_NYU
  .drop_duplicates()
  .sort_values('distance_from_NYU')
  .head(25)
)

### Using the `apply` approach

The `apply` function allow the users to pass a function and apply it on every single row or column of a Pandas datarame.

In [ ]:
!sudo pip3 install -q -U geopy

from geopy import distance

# A bit more accurate distance calculation, which returns back
# the distance in miles. However, we cannot pass a dataframe
# to the function but only individual values
def distance_from_NYU_geodesic(row):
  NYU_lon = -73.9962293
  NYU_lat = 40.7291527
  NYU = (NYU_lat, NYU_lon)
  rest = (row.LATITUDE, row.LONGITUDE)
  #if pd.isnull(row.Latitude) or pd.isnull(row.Longitude):
  #  return None
  return distance.distance(NYU, rest).miles


In [ ]:
# We now create a smaller version of the dataset with just
# the names/address/lon/lat of the restaurants
rest_names_locations = (
    restaurants
    .assign(
      address = combine_address
    )
    .filter(items = ['CAMIS','DBA','address','LONGITUDE', 'LATITUDE'])
    .query(' LONGITUDE==LONGITUDE ') # idiomatic expression for saying IS NOT NULL
    .query(' LATITUDE==LATITUDE ') # idiomatic expression for saying IS NOT NULL
    .drop_duplicates()
)

rest_names_locations

In [ ]:
# We will now apply the function distance_from_NYU_geodesic
# to every row of the dataset:
rest_names_locations.apply(distance_from_NYU_geodesic, axis='columns')


In [ ]:
# We will now save the result into a new column
rest_names_locations['distance_from_NYU']=rest_names_locations.apply(distance_from_NYU_geodesic, axis='columns')

In [ ]:
# Let's see how many restaurants are within half a mile from NYU :)
(
    rest_names_locations
    .query('distance_from_NYU < 0.5')
    .sort_values('distance_from_NYU')
)

# Further dataset exploration

## Restaurant Risk Assessment

* Which restaurants or restaurant chains have a history of repeated, high-scoring violations?
* Can we identify patterns in violation types or frequencies based on cuisine type, location?

## Geographic Analysis of Violations

* Are there specific neighborhoods (NTAs) or zip codes that have a higher prevalence of certain violation types (e.g., pest-related violations)?
* How does the average inspection score vary across different boroughs?
Visualize the distribution of restaurants and their inspection scores on a map of NYC.

## Temporal Analysis of Inspections

* How have violation types changed over time?
Are there seasonal trends in certain violation types?
* What is the average time between inspections for restaurants?

## Restaurant Chain Performance

* How do major restaurant chains compare in terms of average inspection scores and violation rates?
* Are there specific chains that consistently perform better or worse than average?
* Analyze the relationship between the number of locations a chain has and its average inspection score.

